# Affichage des données téléphoniques

## Imports

In [ ]:
import folium
import pandas as pd;
from sklearn.cluster import DBSCAN
import mapUtils
import IHM
import branca
import itertools
import copy
from branca.element import Figure, Html
import webbrowser
from weasyprint import HTML, CSS
import branca
from branca.element import Figure, Html
import folium
from selenium import webdriver
from PIL import Image
import io
from selenium.webdriver.chrome.service import Service
from pathlib import Path
from PIL import Image
import os
from pyproj import Transformer
import numpy as np
import matplotlib.pyplot as plt



## Plotting and saving subplots of maps

In [ ]:
ntechs = len(IHM.technologies)
nprovs = len(IHM.providers)

Fonction pour créer une image à partir d'un fichier html 

In [ ]:
def save_png(html_file, png_file, compressing_factor = 1):      
    # Configurer Selenium WebDriver
    service = Service(executable_path=r'/usr/bin/chromedriver')
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(service=service, options=options)
    driver.set_window_size(2760, 1600)
    # Charger la page HTML dans le navigateur
    driver.get(f"file://{str(Path().absolute())}/{html_file}")

    # Prendre une capture d'écran de l'élément
    screenshot = driver.get_screenshot_as_png()
    driver.quit()

    # Convertir la capture d'écran en image avec Pillow
    image = Image.open(io.BytesIO(screenshot))

    # Compresser l'image
    width, height = image.size
    new_size = (width//compressing_factor, height//compressing_factor)
    image = image.resize(new_size)

    # Enregistrer l'image en PNG
    image.save(png_file)

Traçage des comparaisons des différents opérateurs pour chaque technologie

In [ ]:
for technology in IHM.technologies :
    fig = branca.element.Figure()
    for i, provider in enumerate(IHM.providers) :
        subplot = fig.add_subplot(int(nprovs/2), nprovs-int(nprovs/2), i+1)
        subplot.add_child(mapUtils.getMap(copy.deepcopy(IHM.df), [provider], IHM.regions, [technology], "Opérateurs", location = [46.8, 2.3], zoom_start = 6)) 
    mapUtils.addLegend(fig,{'Bouygues Telecom':'#035afc', 'Orange':'#fc5603', 'SFR':'#169e26', 'Free Mobile':'#dbd640'})
    fig.save(f"../../out/maps/providers-{technology}.html")
    save_png(f"../../out/maps/providers-{technology}.html", f"../../out/images/providers-{technology}.png", 2)

Traçage des répartitions des stations de base selon les opérateurs

In [ ]:
fig = branca.element.Figure()
for i,provider in enumerate(IHM.providers) :
    subplot = fig.add_subplot(int(nprovs/2), nprovs-int(nprovs/2), i+1)
    subplot.add_child(mapUtils.getMap(copy.deepcopy(IHM.df), [provider], IHM.regions, IHM.technologies, "Opérateurs", location = [46.8, 2.3], zoom_start = 6)) 
mapUtils.addLegend(fig,{'Bouygues Telecom':'#035afc', 'Orange':'#fc5603', 'SFR':'#169e26', 'Free Mobile':'#dbd640'})
fig.save("../../out/maps/subplots-operators.html")
save_png("../../out/maps/subplots-operators.html", "../../out/images/subplots-operators.png", 2)

### Couverture ciblée

In [ ]:
data_couv_cible = pd.read_csv("../../database/2023_T4_dcc_Metropole_sites.csv", sep=";", decimal=',')
Nan_values_data_couv_cible = data_couv_cible[data_couv_cible['x_lambert_93'].isnull() | data_couv_cible['y_lambert_93'].isnull()]
data_couv_cible.dropna(axis=0, how='any', subset=['x_lambert_93', 'y_lambert_93'], inplace=True)


In [ ]:
Nan_values_data_couv_cible[['x_lambert_93', 'y_lambert_93']]

In [ ]:
map = folium.Map(location=[46.8, 2.3], zoom_start=7, tiles="Cartodb Positron")

# Définir le transformateur pour la conversion de Lambert 93 à WGS84
transformer = Transformer.from_crs("epsg:2154", "epsg:4326")

for ind, x, y in data_couv_cible[['x_lambert_93', 'y_lambert_93']].itertuples():
    # Convertir les coordonnées Lambert 93 en longitude/latitude
    latitude, longitude = transformer.transform(x, y)
    map.add_child(folium.RegularPolygonMarker(location=[latitude, longitude], fill_opacity=0.2, radius=1.5, popup=f"({x}, {y})"))
map.save("../../out/maps/couverture_ciblee.html")
save_png("../../out/maps/couverture_ciblee.html","../../out/images/couverture_ciblee.png", 2)